In [47]:
import pandas as pd
from glob import glob
import chardet
from tqdm import tqdm
import os
import json
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from models.model import LLM

/workspace/changwoo/DataGeneration/models/model.py:7: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm=HuggingFaceEndpoint(
/root/.cache/pypoetry/virtualenvs/datageneration-Lyt16Uhg-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/changwoo/DataGeneration/models/model.py:6: LangChainDeprecationWarning: The class `ChatHuggingFace` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-h

### Load Dataset

In [58]:
from importlib.resources import files
DATA_PATH = files("data")
RAW_PATH = str(DATA_PATH.joinpath("raw/*.csv"))
PREPROCESSED_PATH = str(DATA_PATH.joinpath("preprocessed/"))

In [56]:
RAW_DATA = sorted(glob(RAW_PATH))

def extract_df(path):
    with open(path, 'rb') as f:
        result = chardet.detect(f.read())
    name, df = os.path.basename(path), pd.read_csv(path, encoding=result['encoding'])
    return name, df

dfs = [extract_df(i) for i in RAW_DATA]

Preprocessing

In [50]:
queries = []
for name, df in tqdm(dfs):
    cols = list(df.columns)
    for _, rows in df.iterrows():
        text = ""
        for row, col in zip(rows, cols):
            text += col + " : " + str(row) + "\n"
        queries.append([name, text])
        text = ""

100%|██████████| 4/4 [00:00<00:00,  9.65it/s]


### Set LangGraph

In [51]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define Call model
def call_model(state: MessagesState):
    response = LLM.invoke(state["messages"])
    return {"messages": response}

# Set Node
workflow.add_node("model", call_model)

# Set Edge
workflow.add_edge(START, "model")

# Set Memory 
memory = MemorySaver()
# graph = workflow.compile(checkpointer=memory)
graph = workflow.compile()

# Set Config
config = {"configurable": {"thread_id": "abc124"}}

(1-1) Prompt for Instruction

In [26]:
# Input
BASE_PROMPT = """
발전소 관련 데이터를 생성하고 있습니다.
용어에 대한 정의를 나타내는 문장을 생성하세요.

예를 들어, 아래와 같은 형식으로 주요 용어를 정의할 수 있습니다:
발전소(Power Plant)는 전기를 생산하기 위해 다양한 에너지원(화석 연료, 원자력, 재생 가능 에너지 등)을 이용하여 전력을 생성하고 이를 전력망에 공급하는 시설을 말한다. 발전소는 사용하는 에너지원과 발전 방식에 따라 여러 종류로 구분된다.

다음 지시사항을 따르세요.
1. 한국어와 영어를 제외한 언어는 사용하지 않습니다.
2. 정확히 아는 정보에 대해서는 자세히 설명합니다.
3. 생성해야할 용어에 대한 정보는 [용어 정보]를 참고하세요.

[용어 정보]
{query}

생성 문장 :
"""

(1-2) Data Generation

In [33]:
SAVE_PATH = os.path.join(PREPROCESSED_PATH,'PowerPlant_Glossary.json')

try:
    dataset = json.load(open(SAVE_PATH, 'r', encoding='utf-8'))
except:
    dataset = []

for idx, query in enumerate(tqdm(queries)):
    PROMPT = BASE_PROMPT.format(query=query[1])
    INPUT_MESSAGES = [SystemMessage(content="당신은 데이터 생성 어시스턴트입니다."), 
                      HumanMessage(PROMPT)]
    try:
        output = graph.invoke({"messages":INPUT_MESSAGES}, config)["messages"][-1].content
        dataset.append({"name": query[0],
                        "info": query[1],
                        "prompt": PROMPT,
                        "response": output})
        with open(SAVE_PATH, 'w', encoding='utf-8') as file:
            json.dump(dataset, file, ensure_ascii=False, indent=4)
    except Exception as e:
        with open(f'error_log.txt', 'a', encoding='utf-8') as file:
            file.write(f"{idx} : {query[1]} : {e}\n")
        continue

100%|██████████| 7108/7108 [7:10:27<00:00,  3.63s/it]   


(2-1) Prompt for Output

In [77]:
# Input
BASE_PROMPT = """
발전소 관련 데이터를 생성하고 있습니다.
다음 답변에 대한 요청문을 생성하세요.

예를 들어 주어진 정보가 있을때, 생성된 요청문은 아래와 같습니다.

# 주어진 정보
태양광발전 판넬(Photovoltaic Panel)은 태양광을 전기 에너지로 변환하는 태양전지 모듈들을 기계적으로 결합하여 널판 형태로 만든 집합체를 말한다. 이 판넬은 어레이 또는 하위의 소어레이에 설치 가능한 단위로 설계되어 미리 조립하고 결선하여 함께 결합된 형태로 제공되며, 현장에서 쉽게 조립할 수 있도록 설계되어 있다.

# 생성된 요청문
태양광발전시스템에 대해 설명해주세요.

다음 지시사항을 따르세요.
1. 한국어와 영어를 제외한 언어는 사용하지 않습니다.
2. 요청문은 간결한 문장 1개만 사용하세요.
3. 요청문은 용어에 대한 정의를 묻는 질문만 사용합니다.
4. 요청문은 다양한 형태를 사용할 수 있습니다. 예를들어, 아래중에 한가지를 사용할 수 있습니다.
    4.1 태양광발전시스템에 대해 설명해주세요.
    4.2 태양광발전시스템이 뭐죠?
    4.3 태양광발전시스템에 대해 알려주실래요?
    4.4 태양광발전시스템이 뭔가요?
    4.5 태양광발전시스템에 대해 알려주세요.

[주어진 정보]
{response}

생성된 요청문 :
"""

(2-2) Data Generation

In [78]:
PREVIOUS_SAVE_PATH = os.path.join(PREPROCESSED_PATH,'PowerPlant_Glossary.json')
NEW_SAVE_PATH = os.path.join(PREPROCESSED_PATH,'PowerPlant_Glossary_ver2.json')
dataset = json.load(open(PREVIOUS_SAVE_PATH, 'r', encoding='utf-8'))

try:
    new_dataset = json.load(open(NEW_SAVE_PATH, 'r', encoding='utf-8'))
except:
    new_dataset = []

for idx, query in enumerate(tqdm(dataset)):
    PROMPT = BASE_PROMPT.format(response=query['response'])
    INPUT_MESSAGES = [SystemMessage(content="당신은 데이터 생성 어시스턴트입니다."), 
                      HumanMessage(PROMPT)]
    try:
        output = graph.invoke({"messages":INPUT_MESSAGES}, config)["messages"][-1].content
        new_dataset.append({"name": query['name'],
                            "info": query['info'],
                            "prompt_for_instruction": PROMPT,
                            "prompt_for_response": query['prompt'],
                            "instruction": output,
                            "response": query['response']})
        with open(NEW_SAVE_PATH, 'w', encoding='utf-8') as file:
            json.dump(new_dataset, file, ensure_ascii=False, indent=4)
    except Exception as e:
        with open(f'error_log.txt', 'a', encoding='utf-8') as file:
            file.write(f"{idx} : {query[1]} : {e}\n")
        continue

100%|██████████| 15444/15444 [2:25:47<00:00,  1.77it/s]  
